# Folium Fun

Let's learn how to map and visualize data using Jupyter Notebooks. Today we are going to work through some examples using Folium and talk about some of the technical details that underlie these code examples. We will learn some more programming tricks along the way as well.

Many of these code snippits are inspired by the Folium examples so if you are looking for additional examples look here:
https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/

We will cover some of the most useful today.

In [ ]:
# Basic start
# Import the folium library/package and some additional subpackages such as features
import folium
from folium import features


# Make a basic folium map
m = folium.Map()

# Display the map
m

In [ ]:
# Change location and zoom
# Larger number == more zoomed in
m = folium.Map(location=[44.971751, -93.243196], zoom_start=40)

# Display the map
m

In [ ]:

m = folium.Map(location=[44.971751, -93.243196], zoom_start=18)

mk = features.Marker([44.971751, -93.243196])

m.add_child(mk)


# Display the map
m


In [ ]:
# Make it a popup

m = folium.Map(location=[44.971751, -93.243196], zoom_start=18)

mk = features.Marker([44.971751, -93.243196])

# Try clicking on me :)
pp = folium.Popup('Hello, I am a pop up!')

mk.add_child(pp)

m.add_child(mk)


# Display the map
m


In [ ]:
# Make another popup

mk = features.Marker([44.972, -93.242])


m.add_child(mk)


# Try clicking on me :)
pp = folium.Popup('Hello, I am a pop up!')


mk.add_child(pp)


# Display the map
m


In [ ]:
# Try to figure this on out from folium Features
# It creates a geojson in code and randomly creates markers.
# Let's break it down.
import numpy as np

N = 1000


#lons=[]
#for num in np.random.normal(size=N)
#    mylon=-93-num
#    lons.append(mylon)

# Here is a nice trick from numpy...
lons = -93 - np.random.normal(size=N,scale=4.0)
lats = 44 - np.random.normal(size=N)


# Let's see an example of a list comprehension (see the code example in 'coordinates' below)
#   (https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions)

data = {
    'type': 'FeatureCollection',
    'features': [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'MultiPoint',
                'coordinates': [[lon, lat] for (lat, lon) in zip(lats, lons)],
            },
            'properties': {'prop0': 'value0'}
        },
    ],
}

m = folium.Map([44, -93], zoom_start=6)
m.add_child(features.GeoJson(data))

#m.save(os.path.join('results', 'Features_6.html'))

m

In [ ]:

# Step 1
mylist = []
for index in range(len(lats)):
    mylist.append([lons[index],lats[index]])

#print(mylist)

mylist2=[]
for (lat, lon) in zip(lats, lons):
    mylist2.append([lon, lat])

#print(mylist2)

mylist3=[]
for (lon, lat) in zip(lons, lats):
    mylist3.append([lon, lat])

print(mylist3)


#[[lon, lat] for (lat, lon) in zip(lats, lons)]

In [ ]:
m = folium.Map(location=[44.971751, -93.243196], zoom_start=18)

my_line = features.ColorLine(
    positions=list([[44.97175, -93.24319],[44.9712, -93.2432]]),
    colors=[3],
    colormap=['y', 'orange'],
    weight=10)

# Add to map (another way)
my_line.add_to(m)

m

In [ ]:
# Cartographic styles by changing base tiles
m = folium.Map(location=[44.971751, -93.243196], zoom_start=18, tiles='cartodbdark_matter')

# Now try some of the others
#OpenStreetMap
#Mapbox Bright
#Stamen Toner
#Mapbox Control Room
# ...
#https://python-graph-gallery.com/288-map-background-with-folium/

m

In [ ]:
# Crazy polygon
m = folium.Map(location=[44.9717, -93.24], zoom_start=14)

locations = [
    [44.9717, -93.2431],
    [44.9718, -93.2420],
    [44.9719, -93.2416],
    [44.9722, -93.2418],
    [44.9743, -93.2423],
]

folium.Polygon(
    locations=locations,
    color='blue',
    weight=6,
    fill_color='red',
    fill_opacity=0.5,
    fill=True,
    popup='A popup!',
    tooltip='Click me!',
).add_to(m)

# Add an icon marker
folium.Marker(
    location=[44.9720, -93.2420],
    popup='Yo',
    icon=folium.Icon(icon='cloud')
).add_to(m)


m

In [ ]:
# From https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Colormaps.ipynb

# Let's take it and do some more stuff with it.

import os
import pandas as pd # We will learn more about pandas later, for now we are using it to open and read a CSV file.
import json
import requests # Used to download data from the internet

# Pull data from the interwebs
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
us_states = f'{url}/us-states.json'
US_Unemployment_Oct2012 = f'{url}/US_Unemployment_Oct2012.csv'

geo_json_data = json.loads(requests.get(us_states).text)
unemployment = pd.read_csv(US_Unemployment_Oct2012)

# If you had these files locally, this is how you would open them up
#us_states = os.path.join('.', 'us-states.json')
#US_Unemployment_Oct2012 = os.path.join(
#    '.', 'US_Unemployment_Oct2012.csv'
#)
#geo_json_data = json.load(open(us_states))
#unemployment = pd.read_csv(US_Unemployment_Oct2012)

unemployment_dict = unemployment.set_index('State')['Unemployment']


def my_color_function(feature):
    """Maps low values to green and hugh values to red."""
    if unemployment_dict[feature['id']] > 6.5:
        return '#0000ff'
    else:
        return '#008000'
    
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'color': 'red',
        'weight': 4,
        'dashArray': '2, 5'
    }
).add_to(m)

# If you want to save your maps you can add this.
#m.save(os.path.join('.', 'MyMap.html'))

m

